# MLFlow + WhyLabs Integration

This tutorial showcases how you can use the whylabs integration to:
* Capture data quality metrics while training a linear regression model in `mlflow`
* Extract whylogs data back into an in-memory format from the MLflow backend
* Log same data back into whylabs platform.

# Getting Started

To run this tutorial:
* Create and then activate the conda environment included using `environment.yml` file and run this notebook using this new environment `whylabs-mlflow` as kernel.
* You'll need to install pip into the conda environment using `conda install pip`

# Setup
First, we want to filter out noisy warnings

In [1]:
import logging
logging.basicConfig(level=logging.INFO)

In [2]:
import os
import random
import datetime
import random
import time
import math
import numpy as np
import pandas as pd
import mlflow
import whylogs
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from whylogs import get_or_create_session
from dotenv import load_dotenv

In [3]:
load_dotenv()

True

In [4]:
assert whylogs.__version__ >= "0.1.13" # we need 0.1.13 or later for MLflow integration

In [5]:
pd.options.mode.chained_assignment = None  # default='warn'

# Enable whylogs Integration

Enable whylogs in MLflow to allow storing whylogs statistical profiles with every run. This method returns `True` if whylogs is able to patch MLflow. You might want to pass a session

In [6]:
session = get_or_create_session(".whylogs.yaml")
whylogs.enable_mlflow(session)

True

# Dataset Preparation

Download and prepare the UCI wine quality dataset. We sample the test dataset further to represent batches of data produced every second.

In [7]:
data = pd.read_csv(os.environ["DATASET_URL"], sep=";")
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [8]:
# Split the data into training and test sets
train, test = train_test_split(data)

In [9]:
# Relocate predicted variable "quality" to y vectors
train_x = train.drop(["quality"], axis=1).reset_index(drop=True)
test_x = test.drop(["quality"], axis=1).reset_index(drop=True)
train_y = train[["quality"]].reset_index(drop=True)
test_y = test[["quality"]].reset_index(drop=True)

## Train a model

Now, we split the data to simulate multiple machine learning model learning looking for the best hyperparameters (simulation of an hyperparameter optimization processs).

First, we split our test sets data to test every algorithm (also, for testing whylogs ability to log dataframes).

In [10]:
# Create an MLflow experiment for our demo
experiment_name = "whylogs demo"
mlflow.set_experiment(experiment_name)

model_params = {"alpha": 1.0,
                "l1_ratio": 0.7}

We can further evaluate the model performance using crossvalidation with 10 folds without overlapping, taking each batch, evaluating the model and then averaging the metrics obtained for each batch.

It's essential to index the data given a timestamp index because information is indexed by timestamp in WhyLabs Platform.

Note that whylogs profiler data is automatically logged when mlflow.end_run() is called implicitly.

In [11]:
n_folds = 10
kf = KFold(n_splits=n_folds)
mae_v = []
for i, (train_index, test_index) in enumerate(kf.split(train_x)):
    with mlflow.start_run(run_name=f"Run {i + 1}"):        
        X_train, X_test = train_x.loc[train_index], train_x.loc[test_index]
        y_train, y_test = train_y.loc[train_index], train_y.loc[test_index]
        
        # Train a model with each split
        lr = ElasticNet(**model_params)
        lr.fit(X_train, y_train)
        print("ElasticNet model (%s):" % model_params)
        
        # Evaluate trained model 
        predicted_output = lr.predict(X_test)
        mae = mean_absolute_error(y_test, predicted_output)

        # Log to mlflow the hyperparameters and evaluation metric
        mlflow.log_params(model_params)
        mlflow.log_metric("mae", mae)
        X_test[0, "mae"] = mae
        
        for k, v in model_params.items():
            train_x.iloc[test_index][k] = v
        
        # use whylogs to log data quality metrics for the current batch
        mlflow.whylogs.log_pandas(
            train_x.iloc[test_index], 
            datetime.datetime.now(datetime.timezone.utc) - datetime.timedelta(days=i)
        )
        print("Subset %.0f, mean absolute error: %s" % (i + 1, mae))
        mae_v.append(mae)


ElasticNet model ({'alpha': 1.0, 'l1_ratio': 0.7}):
Subset 1, mean absolute error: 0.6158390347936468
ElasticNet model ({'alpha': 1.0, 'l1_ratio': 0.7}):
Subset 2, mean absolute error: 0.6297893893637718
ElasticNet model ({'alpha': 1.0, 'l1_ratio': 0.7}):
Subset 3, mean absolute error: 0.7512321484795857
ElasticNet model ({'alpha': 1.0, 'l1_ratio': 0.7}):
Subset 4, mean absolute error: 0.6850834893467308
ElasticNet model ({'alpha': 1.0, 'l1_ratio': 0.7}):
Subset 5, mean absolute error: 0.638706448034305
ElasticNet model ({'alpha': 1.0, 'l1_ratio': 0.7}):
Subset 6, mean absolute error: 0.6414529883419526
ElasticNet model ({'alpha': 1.0, 'l1_ratio': 0.7}):
Subset 7, mean absolute error: 0.6434871133311749
ElasticNet model ({'alpha': 1.0, 'l1_ratio': 0.7}):
Subset 8, mean absolute error: 0.667673330786725
ElasticNet model ({'alpha': 1.0, 'l1_ratio': 0.7}):
Subset 9, mean absolute error: 0.7130783318021062
ElasticNet model ({'alpha': 1.0, 'l1_ratio': 0.7}):
Subset 10, mean absolute error: 